In [154]:
import pandas as pd
import numpy as np
import random
import scipy.optimize as scopt

import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [155]:
from supplement_package.game.gradient import GradientComputation
from supplement_package.game.player import Player
from supplement_package.game.stackelberg import StackelbergPlayer
from supplement_package.game.stackelberg import StackelbergGradientComputation

In [156]:
StackelbergGradientComputation.__dict__

mappingproxy({'__module__': 'supplement_package.game.stackelberg',
              '__init__': <function supplement_package.game.stackelberg.StackelbergGradientComputation.__init__(self) -> None>,
              'risk_utility_grad': <staticmethod at 0x14fb67ee0>,
              'stackelberg_penalty_residual': <staticmethod at 0x14fb67f10>,
              'penalty_jmin': <staticmethod at 0x14fb67f40>,
              'penalty_jmax': <staticmethod at 0x14fb67f70>,
              'penalty_risk_balance': <staticmethod at 0x14fb67fa0>,
              '__doc__': None})

In [157]:
community_size = 3

In [158]:
if community_size == 3:
    D_min = [0.0,0.0,0.0]
    D_max = [10.0,10.0,10.0]

    G_min = [0.0,0.0,0.0]
    G_max = [10.0,0.0,0.0]
    
    Kappa = [[0.0, 10.0, 10.0],
             [10.0, 0.0, 5.0],
             [10.0, 5.0, 0.0]]
    
    Cost = [[0.0, 1.0, 1.0],
            [3.0, 0.0, 1.0],
            [2.0, 1.0, 0.0]]
    
    probabilities = [0.333, 0.333, 0.333]

    #probabilities = [0.5, 0.5]
    connection_matrix = [[0,1,1],[1,0,1],[1,1,0]]

In [159]:
A_tilde = [random.uniform(0,1) for i in range(community_size)]
B_tilde = [random.uniform(0,1) for i in range(community_size)]

a = [random.uniform(0,1) for i in range(community_size)]
b = [random.uniform(0,1) for i in range(community_size)]
d = [random.uniform(0,1) for i in range(community_size)]

d_target = [[random.uniform(0,8) for j in range(len(probabilities))] for i in range(community_size)]
g_res = [[random.uniform(0,3) for j in range(len(probabilities))] for i in range(community_size)]

g_res = np.array(g_res)
d_target = np.array(d_target)

risk_aversion = [random.uniform(0,1) for i in range(community_size)]

In [160]:
agents = []
StackelbergPlayer.community_size = community_size
StackelbergPlayer.probabilities = probabilities

epsilon = 0.001
alpha = [[proba/(1 - risk_aversion[i]) - epsilon for proba in probabilities] for i in range(community_size)]
#alpha = [[0.2 for proba in probabilities] for i in range(community_size)]
gamma = [proba/(1 - min(risk_aversion)) for proba in probabilities]

j_max = [10 for i in range(community_size)]

for i in range(community_size):
    agent = StackelbergPlayer(i, d_target[i], g_res[i], a[i], b[i], d[i], 
                A_tilde[i], B_tilde[i], D_min[i], D_max[i], 
                G_min[i], G_max[i], risk_aversion[i], Kappa[i], Cost[i], connection_matrix[i],
                alpha = alpha[i], 
                gamma = gamma, 
                insurance_bound=j_max[i])
    
    agents.append(agent)

## Gurobi

In [161]:
import gurobipy as gp

In [162]:
def gurobi_add_generation_var(agent, model):

    for proba in agent.probabilities_ind:
        model.addVar(lb = agent.G_min, 
                        ub = agent.G_max, 
                        vtype=gp.GRB.CONTINUOUS, 
                        name = f'G_{agent.id}_{proba}')

    model.update()

In [163]:
def gurobi_add_demand_var(agent, model):

    for proba in agent.probabilities_ind:
        model.addVar(lb = agent.D_min,
                        ub = agent.D_max,
                        vtype = gp.GRB.CONTINUOUS,
                        name = f'D_{agent.id}_{proba}')

    model.update()

In [164]:
def gurobi_add_energy_trading_var(agent, agents, model):

    for agent_2 in agents:
        if agent.connections[agent_2.id]:
            for proba in agent.probabilities_ind:
                    model.addVar(lb = - agent.kappa[agent_2.id],
                                ub = agent.kappa[agent_2.id],
                                vtype = gp.GRB.CONTINUOUS,
                                name = f'q_{agent.id}_{agent_2.id}_{proba}')

    model.update()


In [165]:
def gurobi_add_fin_contracts_var(agent, model):

    for proba in agent.probabilities_ind:
        model.addVar(lb = - float('inf'),
                    ub = float('inf'),
                    vtype = gp.GRB.CONTINUOUS,
                    name = f'W_{agent.id}_{proba}')

    model.update()

In [166]:
def gurobi_add_insurance_var(agent, model):

    for proba in agent.probabilities_ind:
        model.addVar(lb = 0.0,
                    ub = agent.j_max,
                    vtype = gp.GRB.CONTINUOUS,
                    name = f'J_{agent.id}_{proba}')

    model.update()

In [167]:
def gurobi_add_eta_var(agent, model):
    model.addVar(lb = - float('inf'),
                ub = float('inf'),
                vtype = gp.GRB.CONTINUOUS,
                name = f'eta_{agent.id}')

    model.update()

In [168]:
def gurobi_add_residual_var(agent, model):
    
    for proba in agent.probabilities_ind:
        model.addVar(lb = 0,
                    ub = float('inf'),
                    vtype = gp.GRB.CONTINUOUS,
                    name = f'u_{agent.id}_{proba}')

    model.update()

In [169]:
def gurobi_set_objective(agent, model, two_level = True):
    if two_level:
        lExpr = gp.LinExpr()

        for proba_ind, proba in enumerate(agent.probabilities):
            lExpr.add(model.getVarByName(f'W_{agent.id}_{proba_ind}') * gamma[proba_ind] 
                    + model.getVarByName(f'J_{agent.id}_{proba_ind}') * alpha[agent.id][proba_ind]
                    + model.getVarByName(f'u_{agent.id}_{proba_ind}') * proba / (1 - agent.risk_aversion))

        lExpr.add(model.getVarByName(f'eta_{agent.id}'))

    return lExpr

In [170]:
def gurobi_trading_sum_calc(agent, proba, agents, model, weights = False):
    lExpr = gp.LinExpr()

    if weights:
        for agent_2 in agents:
            if agent.connections[agent_2.id]:
                lExpr.add(model.getVarByName(f'q_{agent.id}_{agent_2.id}_{proba}') * agent.trading_cost[agent_2.id])

    else:
        for agent_2 in agents:
            if agent.connections[agent_2.id]:
                lExpr.add(model.getVarByName(f'q_{agent.id}_{agent_2.id}_{proba}'))

    return lExpr

In [171]:
def gurobi_set_SD_balance_constr(agent, agents, model):

    for proba in agent.probabilities_ind:
        model.addConstr(model.getVarByName(f'D_{agent.id}_{proba}')
                        - model.getVarByName(f'G_{agent.id}_{proba}')
                        - agent.G_res[proba]
                        - gurobi_trading_sum_calc(agent, proba, agents, model, weights=False) == 0,
                        name= f'SD balance for agent {agent.id} proba {proba}')

    model.update()

In [172]:
def gurobi_set_bilateral_trading_constr(agent, agents, model):
    for agent_2 in agents:
        if agent.connections[agent_2.id]:
            for proba in agent.probabilities_ind:
                model.addConstr(model.getVarByName(f'q_{agent.id}_{agent_2.id}_{proba}')  
                                + model.getVarByName(f'q_{agent_2.id}_{agent.id}_{proba}') == 0, 
                                name = f'Bilateral trading for pair ({agent.id}, {agent_2.id}) proba {proba}')

    model.update()

In [173]:
def gurobi_quadr_generation(agent, proba, model):
    qExpr_G = gp.QuadExpr()

    qExpr_G.add(0.5 * agent.a 
                * model.getVarByName(f'G_{agent.id}_{proba}') * model.getVarByName(f'G_{agent.id}_{proba}') 
                + agent.b 
                * model.getVarByName(f'G_{agent.id}_{proba}') 
                + agent.d)
    
    return qExpr_G

In [174]:
def gurobi_quadr_demand(agent, proba, model):
    qExpr_D = gp.QuadExpr()

    qExpr_D.add(agent.a_tilde 
                * (model.getVarByName(f'D_{agent.id}_{proba}') - agent.D_target[proba]) 
                * (model.getVarByName(f'D_{agent.id}_{proba}') - agent.D_target[proba]) 
                - agent.b_tilde)

    return qExpr_D

In [175]:
def gurobi_set_residual_constr(agent, agents, model):
    
    for proba in agent.probabilities_ind:
        qExpr = gp.QuadExpr()
        lExpr = gp.LinExpr()
        
        qExpr.add(model.getVarByName(f'D_{agent.id}_{proba}'))
        
        qExpr.add(gurobi_quadr_generation(agent, proba, model))
        qExpr.add(gurobi_quadr_demand(agent, proba, model))
        lExpr.add(gurobi_trading_sum_calc(agent, proba, agents, model, weights=True))

        lExpr.add(- model.getVarByName(f'u_{agent.id}_{proba}') )
        lExpr.add(- model.getVarByName(f'eta_{agent.id}'))
        lExpr.add(- model.getVarByName(f'W_{agent.id}_{proba}'))
        lExpr.add(- model.getVarByName(f'J_{agent.id}_{proba}'))

        model.addConstr(qExpr + lExpr  <= 0,
                        name=f'Residual constraint for agent {agent.id} proba {proba}')
        
        model.update()
    


In [176]:
def gurobi_set_risk_trading_constr(agents, model):
    for proba in agents[0].probabilities_ind:
        lExpr = gp.LinExpr()

        for agent in agents:
            lExpr.add(model.getVarByName(f'W_{agent.id}_{proba}'))

        model.addConstr(lExpr == 0,
                        name = f'Risk trading balance for proba {proba}')

    model.update()

In [177]:
def build_model(agents, model, centralized = True):
    if centralized:
        for agent in agents:
            gurobi_add_demand_var(agent, model)
            gurobi_add_generation_var(agent, model)
            gurobi_add_energy_trading_var(agent, agents, model)
            gurobi_add_eta_var(agent, model)
            gurobi_add_fin_contracts_var(agent, model)
            gurobi_add_insurance_var(agent, model)
            gurobi_add_residual_var(agent, model)

        for agent in agents:
            gurobi_set_bilateral_trading_constr(agent, agents, model)
            gurobi_set_residual_constr(agent, agents, model)
            gurobi_set_SD_balance_constr(agent, agents, model)
            
        gurobi_set_risk_trading_constr(agents, model)

        obj = gp.LinExpr()
        for agent in agents:
            obj.add(gurobi_set_objective(agent, model, two_level=True))

        model.setObjective(obj, gp.GRB.MINIMIZE)
        model.update()

In [178]:
model_1 = gp.Model()
build_model(agents, model_1)

model_1.getQConstrs()

[<gurobi.QConstr Residual constraint for agent 0 proba 0>,
 <gurobi.QConstr Residual constraint for agent 0 proba 1>,
 <gurobi.QConstr Residual constraint for agent 0 proba 2>,
 <gurobi.QConstr Residual constraint for agent 1 proba 0>,
 <gurobi.QConstr Residual constraint for agent 1 proba 1>,
 <gurobi.QConstr Residual constraint for agent 1 proba 2>,
 <gurobi.QConstr Residual constraint for agent 2 proba 0>,
 <gurobi.QConstr Residual constraint for agent 2 proba 1>,
 <gurobi.QConstr Residual constraint for agent 2 proba 2>]

In [197]:
model_1.getQCRow(model_1.getQConstrs()[6])

<gurobi.QuadExpr: -8.391569359499917 D_1_2 + 0.2452439762807619 G_1_2 + 3.0 q_1_0_2 + q_1_2_2 + -1.0 eta_1 + -1.0 W_1_2 + -1.0 J_1_2 + -1.0 u_1_2 + [ 0.9887584561452162 D_1_2 ^ 2 + 0.33566421997887563 G_1_2 ^ 2 ]>

In [180]:
model_1.getConstrs()

[<gurobi.Constr Bilateral trading for pair (0, 1) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 2>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 2>,
 <gurobi.Constr SD balance for agent 0 proba 0>,
 <gurobi.Constr SD balance for agent 0 proba 1>,
 <gurobi.Constr SD balance for agent 0 proba 2>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 0>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 1>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 2>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 0>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 1>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 2>,
 <gurobi.Constr SD balance for agent 1 proba 0>,
 <gurobi.Constr SD balance for agent 1 proba 1>,
 <gurobi.Constr SD balance for agen

In [181]:
model_1.getConstrs()

[<gurobi.Constr Bilateral trading for pair (0, 1) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 2>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 2>,
 <gurobi.Constr SD balance for agent 0 proba 0>,
 <gurobi.Constr SD balance for agent 0 proba 1>,
 <gurobi.Constr SD balance for agent 0 proba 2>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 0>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 1>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 2>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 0>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 1>,
 <gurobi.Constr Bilateral trading for pair (1, 2) proba 2>,
 <gurobi.Constr SD balance for agent 1 proba 0>,
 <gurobi.Constr SD balance for agent 1 proba 1>,
 <gurobi.Constr SD balance for agen

In [182]:
model_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 30 rows, 66 columns and 81 nonzeros
Model fingerprint: 0x2992630f
Model has 9 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-01, 1e+00]
  QLMatrix range   [2e-01, 1e+01]
  Objective range  [9e-01, 9e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e-01, 3e+00]
  QRHS range       [1e+00, 4e+01]
Presolve removed 9 rows and 6 columns
Presolve time: 0.00s
Presolved: 63 rows, 78 columns, 174 nonzeros
Presolved model has 9 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 12
 AA' NZ     : 1.770e+02
 Factor NZ  : 6.310e+02
 Factor Ops : 7.769e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.638135

In [104]:
model_1.getVars()

[<gurobi.Var D_0_0 (value 1.1149824107886)>,
 <gurobi.Var D_0_1 (value 2.4743425930397582)>,
 <gurobi.Var D_0_2 (value 1.0218057228397335e-06)>,
 <gurobi.Var G_0_0 (value 1.9788831067197437)>,
 <gurobi.Var G_0_1 (value 0.21368037920749133)>,
 <gurobi.Var G_0_2 (value 0.5009337013158396)>,
 <gurobi.Var q_0_1_0 (value -1.00291488527626)>,
 <gurobi.Var q_0_1_1 (value -0.23084151369975636)>,
 <gurobi.Var q_0_1_2 (value -1.3348624671282145)>,
 <gurobi.Var eta_0 (value -0.31245751976431835)>,
 <gurobi.Var W_0_0 (value 0.0)>,
 <gurobi.Var W_0_1 (value 0.0)>,
 <gurobi.Var W_0_2 (value 0.0)>,
 <gurobi.Var J_0_0 (value 0.0)>,
 <gurobi.Var J_0_1 (value 0.0)>,
 <gurobi.Var J_0_2 (value 0.0)>,
 <gurobi.Var u_0_0 (value 3.934966570845231)>,
 <gurobi.Var u_0_1 (value 4.007242157868555)>,
 <gurobi.Var u_0_2 (value 1.435075960707353e-07)>,
 <gurobi.Var D_1_0 (value 4.0958700256853025)>,
 <gurobi.Var D_1_1 (value 2.36675269765076)>,
 <gurobi.Var D_1_2 (value 5.272833893820205)>,
 <gurobi.Var G_1_0 (valu

In [96]:
def gurobi_model(agents, model, alpha, gamma):
    qExpr_G = gp.QuadExpr()
    qExpr_D = gp.QuadExpr()
    lExpr_q = gp.LinExpr()
    lExpr_W = gp.LinExpr()
    lExpr_J = gp.LinExpr()
    lExpr_u = gp.LinExpr()
    lExpr_obj = gp.LinExpr()

    for agent in agents:
        eta = model.addVar(lb = - float('inf'),
                        ub = float('inf'),
                        vtype = gp.GRB.CONTINUOUS,
                        name = f'eta_{agent.id}')

        for proba in agent.probabilities_ind:
            G = model.addVar(lb = agent.G_min, 
                            ub = agent.G_max, 
                            vtype=gp.GRB.CONTINUOUS, 
                            name = f'G_{agent.id}_{proba}')

            qExpr_G.add(agent.a * G * G + agent.b * G + agent.d)

            D = model.addVar(lb = agent.D_min,
                            ub = agent.D_max,
                            vtype = gp.GRB.CONTINUOUS,
                            name = f'D_{agent.id}_{proba}')

            qExpr_D.add(agent.a_tilde * (D - agent.D_target[proba]) * (D - agent.D_target[proba]) + agent.b_tilde)

            W = model.addVar(lb = -5,
                        ub = 5,
                        vtype = gp.GRB.CONTINUOUS,
                        name = f'W_{agent.id}_{proba}')
        
            lExpr_W.add(W * gamma[proba])

            J = model.addVar(lb = 0.0,
                        ub = agent.j_max,
                        vtype = gp.GRB.CONTINUOUS,
                        name = f'J_{agent.id}_{proba}')

            lExpr_J.add(J * alpha[agent.id][proba])

            sum_q = gp.LinExpr()
            for player in agents:
                q = (
                    model.addVar(lb = - agent.kappa[player.id],
                                ub = agent.kappa[player.id],
                                vtype = gp.GRB.CONTINUOUS,
                                name = f'q_{agent.id}_{player.id}_{proba}') if agent.connections[player.id] 
                    else 
                        model.addVar(lb = 0,
                                ub = 0,
                                vtype = gp.GRB.CONTINUOUS,
                                name = f'q_{agent.id}_{player.id}_{proba}')
                    )

                lExpr_q.add(agent.trading_cost[player.id] * q)
                sum_q.add(q)

            model.addConstr(D == G + agent.G_res[proba] + sum_q, 
                            name= f'SD balance for agent {agent.id} proba {proba}')

            u = model.addVar(lb = 0,
                            ub = float('inf'),
                            vtype = gp.GRB.CONTINUOUS,
                            name = f'u_{agent.id}_{proba}')

            lExpr_u.add(u * agent.probabilities[proba] / (1 - agent.risk_aversion))

            lExpr_obj.add(lExpr_J + lExpr_W + lExpr_u)

            model.addConstr(qExpr_G + qExpr_D + lExpr_q - eta - u <= 0, 
                            name=f'Residual constraint for agent {agent.id} proba {proba}')

        lExpr_obj.add(eta)

    model.setObjective(lExpr_obj, gp.GRB.MINIMIZE)
    model.update()


In [97]:
def gurobi_bilateral_trading(agents, model):
    for agent in agents:
        for agent_2 in agents:
            if agent.connections[agent_2.id]:
                for proba in agent.probabilities_ind:

                    model.addConstr(model.getVarByName(f'q_{agent.id}_{agent_2.id}_{proba}')  
                                    + model.getVarByName(f'q_{agent_2.id}_{agent.id}_{proba}') == 0, 
                                    name = f'Bilateral trading for pair ({agent.id}, {agent_2.id}) proba {proba}')

    model.update()

In [98]:
def risk_trading_constraint(agents, model, probabilities):
    risk_sum = gp.LinExpr()
    for proba, _ in enumerate(probabilities):
        for agent in agents:
                risk_sum.add(model.getVarByName(f'W_{agent.id}_{proba}'))

        model.addConstr(risk_sum == 0, name= 'Risk trading balance')

    model.update()

In [99]:
model_2 = gp.Model()
gurobi_model(agents, model_2, alpha=alpha, gamma=gamma)
gurobi_bilateral_trading(agents, model_2)
risk_trading_constraint(agents, model_2, probabilities)

In [100]:
model_2.getConstrs()

[<gurobi.Constr SD balance for agent 0 proba 0>,
 <gurobi.Constr SD balance for agent 0 proba 1>,
 <gurobi.Constr SD balance for agent 0 proba 2>,
 <gurobi.Constr SD balance for agent 1 proba 0>,
 <gurobi.Constr SD balance for agent 1 proba 1>,
 <gurobi.Constr SD balance for agent 1 proba 2>,
 <gurobi.Constr SD balance for agent 2 proba 0>,
 <gurobi.Constr SD balance for agent 2 proba 1>,
 <gurobi.Constr SD balance for agent 2 proba 2>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 1) proba 2>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 0>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 1>,
 <gurobi.Constr Bilateral trading for pair (0, 2) proba 2>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 0>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 1>,
 <gurobi.Constr Bilateral trading for pair (1, 0) proba 2>,
 <gurobi.Constr Bil

In [ ]:
model_1.getVarByName('W_0_2')

## AMPL

In [183]:
from amplpy import AMPL, Environment, DataFrame

ampl = AMPL(Environment('/Users/ishilov/Documents/ampl.macos64'))

ampl.read('/Users/ishilov/Documents/risk_paper/risk_paper/ampl/3_nodes_centr_risk.mod')
ampl.readData('/Users/ishilov/Documents/risk_paper/risk_paper/ampl/3_nodes_centr_risk.dat')


In [184]:
nodes = ['n_0', 'n_1', 'n_2']

In [185]:
Cost_dict = {}
for i in range(community_size):
    for j in range(community_size):
        Cost_dict.update({(nodes[i],nodes[j]):Cost[i][j]})

Kappa_dict = {}
for i in range(community_size):
    for j in range(community_size):
        Kappa_dict.update({(nodes[i],nodes[j]):Kappa[i][j]})

In [186]:
d_target_dict = {}
g_res_dict = {}
alpha_dict = {}
for i in range(community_size):
    for j in range(len(probabilities)):
        d_target_dict.update({('n_{}'.format(i), 'p_{}'.format(j)): d_target[i][j]})
        g_res_dict.update({('n_{}'.format(i), 'p_{}'.format(j)): g_res[i][j]})
        alpha_dict.update({('n_{}'.format(i), 'p_{}'.format(j)): alpha[i][j]})

In [187]:
for agent in agents:
    print(agent.alpha)

[8.716637297981197, 8.716637297981197, 8.716637297981197]
[8.127341667060136, 8.127341667060136, 8.127341667060136]
[0.9381766573948582, 0.9381766573948582, 0.9381766573948582]


In [188]:
alpha_dict

{('n_0', 'p_0'): 8.716637297981197,
 ('n_0', 'p_1'): 8.716637297981197,
 ('n_0', 'p_2'): 8.716637297981197,
 ('n_1', 'p_0'): 8.127341667060136,
 ('n_1', 'p_1'): 8.127341667060136,
 ('n_1', 'p_2'): 8.127341667060136,
 ('n_2', 'p_0'): 0.9381766573948582,
 ('n_2', 'p_1'): 0.9381766573948582,
 ('n_2', 'p_2'): 0.9381766573948582}

In [189]:
p_dict = {}
for j in range(len(probabilities)):
    p_dict.update({'p_{}'.format(j) : probabilities[j]})

In [190]:
if community_size == 3:

    ampl.getParameter('a').setValues(a)
    ampl.getParameter('b').setValues(b)
    ampl.getParameter('d').setValues(d)
    ampl.getParameter('a_t').setValues(A_tilde)
    ampl.getParameter('b_t').setValues(B_tilde)

    ampl.getParameter('gamma').setValues(gamma)
    ampl.getParameter('alpha').setValues(alpha_dict)
    ampl.getParameter('J_max').setValues(j_max)

    ampl.getParameter('D_max').setValues(D_max)
    ampl.getParameter('G_max').setValues(G_max)
    ampl.getParameter('D_min').setValues(D_min)
    ampl.getParameter('G_min').setValues(G_min)

    ampl.getParameter('chi').setValues(risk_aversion)
    
    ampl.getParameter('cost').setValues(Cost_dict)

    ampl.getParameter('kappa').setValues(Kappa_dict)
    
    ampl.getParameter('p').setValues(p_dict)
                
    ampl.getParameter('D_t').setValues(d_target_dict)
    ampl.getParameter('G_d').setValues(g_res_dict)
    
    # Solve
    ampl.setOption('solver', 'MINOS')
    ampl.solve()

MINOS 5.51: optimal solution found.
168 iterations, objective 7.945591432
Nonlin evals: constrs = 366, Jac = 365.


In [191]:
if community_size ==3:
    g = ampl.getVariable('G').getValues().toDict()
    d = ampl.getVariable('D').getValues().toDict()
    u = ampl.getVariable('u').getValues().toDict()
    eta = ampl.getVariable('eta').getValues().toDict()
    q = ampl.getVariable('quant').getValues().toDict()
    Q = ampl.getVariable('Q').getValues().toDict()
    j = ampl.getVariable('J').getValues().toDict()
    w = ampl.getVariable('W').getValues().toDict()


In [192]:
ampl.display('quant')

quant :=
n_0 n_1 p_0    2.89132
n_0 n_1 p_1    2.05948
n_0 n_1 p_2    2.35859
n_0 n_2 p_0   -5.93566
n_0 n_2 p_1   -2.22921
n_0 n_2 p_2   -6.75393
n_1 n_0 p_0   -2.89132
n_1 n_0 p_1   -2.05948
n_1 n_0 p_2   -2.35859
n_1 n_2 p_0    5
n_1 n_2 p_1    2.26107
n_1 n_2 p_2    5
n_2 n_0 p_0    5.93566
n_2 n_0 p_1    2.22921
n_2 n_0 p_2    6.75393
n_2 n_1 p_0   -5
n_2 n_1 p_1   -2.26107
n_2 n_1 p_2   -5
;



In [ ]:
w

In [ ]:
def assign_ampl(d, g, eta, u, q, w, J, agents):
    """Assigns the solution from AMPL to agents"""

    for i in range(community_size):
        id = 'n_{}'.format(i)

        for j in range(len(probabilities)):
            proba = 'p_{}'.format(j)
            agents[i].D[j] = d[(id, proba)]

    for i in range(community_size):
        id = 'n_{}'.format(i)

        for j in range(len(probabilities)):
            proba = 'p_{}'.format(j)
            agents[i].w[j] = w[(id, proba)]

    for i in range(community_size):
        id = 'n_{}'.format(i)

    for j in range(len(probabilities)):
        proba = 'p_{}'.format(j)
        agents[i].j[j] = J[(id, proba)]

    for i in range(community_size):
        id = 'n_{}'.format(i)

    for j in range(len(probabilities)):
        proba = 'p_{}'.format(j)
        agents[i].G[j] = g[(id, proba)]

    for i in range(community_size):
        id = 'n_{}'.format(i)

        for j in range(len(probabilities)):
            proba = 'p_{}'.format(j)
            agents[i].u[j] = u[(id, proba)]

    for i in range(community_size):
        id = 'n_{}'.format(i)

        agents[i].eta = eta[id]

    for i in range(community_size):
        id_1 = 'n_{}'.format(i)

        for j in range(community_size):
            id_2 = 'n_{}'.format(j)

            for k in range(len(probabilities)):
                proba = 'p_{}'.format(k)

                agents[i].q[j][k] = q[(id_1, id_2, proba)]

In [ ]:
assign_ampl(d, g, eta, u, q, w, j, agents)

## Gradient

In [ ]:
Player.probabilities

In [ ]:
agents[0].__dict__

In [ ]:
for i in range(community_size):
    print('Utility update:')
    print(StackelbergGradientComputation.utility_grad(agents[i]))
    print('-------')
    print('D_min bound')
    print(StackelbergGradientComputation.penalty_dmin(agents[i]))
    print('-------')
    print('D_max bound')
    print(StackelbergGradientComputation.penalty_dmax(agents[i]))
    print('-------')
    print('G_min bound')
    print(StackelbergGradientComputation.penalty_gmin(agents[i]))
    print('-------')
    print('G_max bound')
    print(StackelbergGradientComputation.penalty_gmax(agents[i]))
    print('-------')
    print('Penalty supply-demand balance bound')
    print(StackelbergGradientComputation.penalty_balance(agents[i]))
    print('-------')
    print('U bound')
    print(StackelbergGradientComputation.penalty_u(agents[i]))
    print('-------')
    print('Penalty trading bound')
    print(StackelbergGradientComputation.penalty_trading_bound(agents[i]))
    print('-------')
    print('Bilateral trading bound')
    print(StackelbergGradientComputation.penalty_bilateral_trading(agents[i], agents))
    print('-------')
    print('Penalty residual bound')
    print(StackelbergGradientComputation.penalty_residual(agents[i]))
    print('----------------------------------------------')

In [ ]:
mu = 0.003
rho = 2000
k = 0

plot_penalty_residual = [[[] for j in range(len(probabilities))] for i in range(community_size)]
plot_balance = [[[] for j in range(len(probabilities))] for i in range(community_size)]

min_ra = min(risk_aversion)
while k <= 1000:
    #print(k)
    #print('--------------------------')
    for agent in agents:
        #Update of grad_d of the agent

        agent.grad_D = rho*(StackelbergGradientComputation.penalty_dmin(agent) 
                            + StackelbergGradientComputation.penalty_dmax(agent)
                            + StackelbergGradientComputation.penalty_balance(agent)['update_d']
                            + StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_d'])

        agent.grad_G = rho*(StackelbergGradientComputation.penalty_gmin(agent) 
                            + StackelbergGradientComputation.penalty_gmax(agent)
                            + StackelbergGradientComputation.penalty_balance(agent)['update_g']
                            + StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_g'])

        agent.grad_eta = (StackelbergGradientComputation.risk_utility_grad(agent, min_ra)['update_eta']
                            + rho*StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_eta'])

        agent.grad_u = (StackelbergGradientComputation.risk_utility_grad(agent, min_ra)['update_u']
                            + rho*StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_u']
                            + rho*StackelbergGradientComputation.penalty_u(agent))



        agent.grad_q = rho*(StackelbergGradientComputation.penalty_trading_bound(agent)
                            + StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_q']
                            + StackelbergGradientComputation.penalty_balance(agent)['update_q']
                            + StackelbergGradientComputation.penalty_bilateral_trading(agent, agents))

        agent.grad_w = (StackelbergGradientComputation.risk_utility_grad(agent, min_ra)['update_w']
                            + rho * StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_w']
                            + rho * StackelbergGradientComputation.penalty_risk_balance(agents))

        agent.grad_j = (StackelbergGradientComputation.risk_utility_grad(agent, min_ra)['update_j']
                            + rho * StackelbergGradientComputation.stackelberg_penalty_residual(agent)['update_j']
                            + rho * StackelbergGradientComputation.penalty_jmin(agent)
                            + rho * StackelbergGradientComputation.penalty_jmax(agent))
        

        #print('Agent {} variables'.format(agent.id))
        #print(agent.variables_to_vector())
        #print('--------------------------------')

    #Agent's variables update
    for agent in agents:

        N = k+1000

        agent.D = agent.D - mu/(N)*agent.grad_D
        agent.G = agent.G - mu/(N)*agent.grad_G
        agent.eta = agent.eta - mu/(N)*agent.grad_eta
        agent.u = agent.u - mu/(N)*agent.grad_u
        agent.q = agent.q - mu/(N)*agent.grad_q
        agent.w = agent.w - mu/(N)*agent.grad_w
        agent.j = agent.j - mu/(N)*agent.grad_j

        agent.plot_eta.append(agent.eta)

        for i in range(len(probabilities)):
            agent.plot_d[i].append(agent.D[i])
            agent.plot_g[i].append(agent.G[i])
            agent.plot_u[i].append(agent.u[i])
            agent.plot_w[i].append(agent.w[i])
            agent.plot_j[i].append(agent.j[i])

            plot_penalty_residual[agent.id][i].append(StackelbergGradientComputation.stackelberg_penalty_residual(agent)['violation'])
            plot_balance[agent.id][i].append(StackelbergGradientComputation.penalty_balance(agent)['violation'])

            for agent_2 in agents:
                agent.plot_q[agent_2.id][i].append(agent.q[agent_2.id][i])
    
    

    k +=1

In [ ]:
for agent in agents:
    for i in range(len(probabilities)):
        plt.plot(agent.plot_d[i])
    
    plt.show()

In [ ]:
for agent in agents:
    for i in range(len(probabilities)):
        plt.plot(agent.plot_w[i])
    
    plt.show()

In [ ]:
for agent in agents:
    for i in range(len(probabilities)):
        plt.plot(agent.plot_j[i])
    
    plt.show()

In [ ]:
for agent in agents:
    for i in range(len(probabilities)):
        plt.plot(plot_penalty_residual[agent.id][i])
    
    plt.show()


In [ ]:
for agent in agents:
    for i in range(len(probabilities)):
        plt.plot(plot_balance[agent.id][i])
    
    plt.show()